In [0]:
import numpy as np                                                                                                
import tensorflow as tf

In [10]:
class HMM(object):
    def __init__(self, initial_prob, trans_prob, obs_prob):
        self.N = np.size(initial_prob)
        self.initial_prob = initial_prob
        self.trans_prob = trans_prob
        self.emission = tf.constant(obs_prob)

        
        self.obs_idx = tf.placeholder(tf.int32,name="obs_idx")
        self.fwd = tf.placeholder(tf.float64,name="fwd")
        self.viterbi = tf.placeholder(tf.float64,name="viterbi")
        
    def get_emission(self, obs_idx):
        slice_location = [0, obs_idx]
        num_rows = tf.shape(self.emission)[0]
        slice_shape = [num_rows, 1]#num_hidden_states X 1
        return tf.transpose(tf.slice(self.emission, slice_location, slice_shape))#1 X num_hidden_states

    def forward_init_op(self):
        obs_prob = self.get_emission(self.obs_idx)#1 X num_hidden_states
        fwd = tf.multiply(self.initial_prob, obs_prob)#1 X num_hidden_states mul 1 X num_hidden_states
        #initial_prob - 1 X num_hidden_states obs_probs is also 1 X num_hidden_states
        #fwd = tf.transpose(fwd)
        stateIndex = tf.argmax(fwd,1)
        viterbi = tf.transpose(fwd)
        return fwd,stateIndex,viterbi

    def forward_op(self):
        tensorProbHiddenStates = tf.matmul(self.fwd,self.trans_prob)#matmul 1 X num_hidden_states to num_hidden_states X num_hidden_states - 1 X num_hidden_states result
        fwd = tensorProbHiddenStates * self.get_emission(self.obs_idx)#1 X num_hidden_states * 1 X num_hidden_states - 1 X num_hidden_states
        obs_probs = self.get_emission(self.obs_idx) # shape is 1 X num_hidden_states
        viterbi_trans_mul = tf.multiply(self.viterbi,self.trans_prob) # broacast of self.viterbi
        viterbi_new_prob_scores = tf.multiply(viterbi_trans_mul,obs_probs) #broadcast of obs_probs
        viterbi_max_scores = tf.reduce_max(viterbi_new_prob_scores,0,keepdims=True)
        viterbi_max_indices = tf.argmax(viterbi_max_scores,1)
        viterbi = tf.transpose(viterbi_max_scores)
        stateIndex = viterbi_max_indices
        return fwd,viterbi_max_scores,viterbi_trans_mul,viterbi_new_prob_scores,viterbi_max_indices,viterbi,stateIndex


def forward_algorithm(sess, hmm, observations):
    stateIndexArray = np.empty((len(observations),1),np.int64)
    stateIndexArray.fill(-1)
    currentFillIndex = 0
    fwd,stateIndex,viterbi = sess.run(hmm.forward_init_op(), feed_dict={hmm.obs_idx: observations[0]})
    stateIndexArray[currentFillIndex][0] = stateIndex[0]
    currentFillIndex += 1
    for t in range(1, len(observations)):
        fwd,vit,trans_mul,new_scores,max_index_viterbi,viterbi,stateIndex = sess.run(hmm.forward_op(), feed_dict={hmm.obs_idx: observations[t], hmm.fwd: fwd,hmm.viterbi : viterbi})
        print(viterbi)
        stateIndexArray[currentFillIndex][0] = stateIndex[0]
        currentFillIndex += 1
    prob = sess.run(tf.reduce_sum(fwd))
    return prob,stateIndexArray

if __name__ == '__main__':
    initial_prob = np.array([[0.6], [0.4]])#num_hidden_states X 1
    initial_prob = tf.transpose(initial_prob)#1 X num_hidden_states
    trans_prob = np.array([[0.7, 0.3], [0.4, 0.6]]) # num_hidden_states X num_hidden_states
    obs_prob = np.array([[0.5, 0.4, 0.1], [0.1, 0.3, 0.6]]) # num_hidden_states X num_observables
    
    hmm = HMM(initial_prob=initial_prob, trans_prob=trans_prob, obs_prob=obs_prob)

    #observations = [0, 1, 1, 2, 1]
    #observations = [0, 1, 1,2,1,2,2,2,1]
    #observations = [2,2,0]
    #observations = [0, 1, 1, 2,1]
    observations = [0,1,1,2,1,0,0,0,1,2,2,1]
    with tf.Session() as sess:
        prob,hiddenStateSeq = forward_algorithm(sess, hmm, observations)
        print('Probability of observing {} is {}'.format(observations, prob))
        print("Hidden State Sequence Id Is :: "+str(hiddenStateSeq))
                                                                                                  

[[0.084]
 [0.027]]
[[0.02352]
 [0.00756]]
[[0.0016464]
 [0.0042336]]
[[0.00067738]
 [0.00076205]]
[[2.370816e-04]
 [4.572288e-05]]
[[8.297856e-05]
 [7.112448e-06]]
[[2.9042496e-05]
 [2.4893568e-06]]
[[8.13189888e-06]
 [2.61382464e-06]]
[[5.69232922e-07]
 [1.46374180e-06]]
[[5.85496719e-08]
 [5.26947047e-07]]
[[8.43115276e-08]
 [9.48504685e-08]]
Probability of observing [0, 1, 1, 2, 1, 0, 0, 0, 1, 2, 2, 1] is 2.7176685317787254e-06
Hidden State Sequence Id Is :: [[0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]]
